In [1]:
from explaneat.data.uci import UCI_WRANGLER
from explaneat.experimenter.experiment import GenericExperiment
from explaneat.evaluators.evaluators import binary_cross_entropy

import torch
import torch.nn as nn
import torch.optim as optim

import logging
import random

import os

import neat

from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode

from explaneat.experimenter.results import Result, ResultsDatabase

from copy import deepcopy

from datetime import datetime

import pandas as pd

import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

ModuleNotFoundError: No module named 'explaneat'

In [2]:
experiment_config_file = './experiment_config.json'
experiment = GenericExperiment(experiment_config_file, confirm_path_creation=False)
logger = experiment.logger

NameError: name 'GenericExperiment' is not defined

In [ ]:

USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")
logger.info("Using device: {}".format(device))


# BC Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [ ]:
random.seed(experiment.config["random_seed"])
logger.info("random.seed set to {}".format(experiment.config["random_seed"]))

## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [ ]:
data_wrangler = UCI_WRANGLER(experiment.config['data']['raw_location'],
        experiment.config['data']['raw_data_meta'])

In [ ]:
data_wrangler.create_train_test_split(experiment.config["train_test_ratio"],
            experiment.config["random_seed"])
data_wrangler.send_train_test_to_device(device)

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `CrossEntropyLoss` from `PyTorch`

## Base configuration

We are going to create the base configuration according to an external configuration file. Per experiment, we will adjust this, later, but this defines the defaults across all runs.

In [ ]:
config_path = "./config-bchard"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)


In [ ]:
experiment.register_config_file("./config-bchard", "neat_config")

We also want to put a hard limit on how long this can go on for.

We will create a method to manage the instantiation of a population on the basis of a specific config.

In [ ]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.BCELoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

# Experiment 1: Vary population size

The first experiment is going to examine the difference in run time different population sizes. 

In [ ]:
epoch_points = [10, 25, 50, 100, 150]
# epoch_points = [10]

In [ ]:
base_config.pop_size

In [ ]:
saveLocationTemplate = './../../data/experiments/bchard/experiment-test-{}-{}/'

## Start the experiment

In [ ]:
len(data_wrangler.X_train.shape)

In [ ]:
data_wrangler.ys.dtype

In [ ]:
my_random_seed = experiment.config["random_seed"]
for epochs in epoch_points:
    for iteration_no in range(5):
        my_random_seed += 1
        random.seed(my_random_seed)
        start_time = datetime.now()
        
        logger.info("################################################")
        logger.info("################################################")
        logger.info("Starting epochs {} iteration {}".format(epochs, iteration_no))
        logger.info("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        logger.info("################################################")
        logger.info("################################################")
        
        
        config = deepcopy(base_config)
#         config.pop_size = pop_size
        
        saveLocation = saveLocationTemplate.format(epochs, iteration_no)
        
        p = instantiate_population(config, data_wrangler.X_train, data_wrangler.y_train, saveLocation)
        # Run for up to nGenerations generations.
        winner = p.run(binary_cross_entropy, experiment.config["max_n_generations"], nEpochs = epochs)
        
        g = p.best_genome

        
        end_time = datetime.now()
        
        p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  
        
        winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

        results = []
        for xi, xo in zip(data_wrangler.X_test, data_wrangler.y_test):
            output = winner_net.activate(xi)
            results.append([xi, xo, output])

        df = pd.DataFrame(results)
        df.to_csv(os.path.join(saveLocation, 'results.csv'))

        ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

        ancestors = {
            k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
        }

        ## Save all of these to disc
        filename = 'fullStatus.xplnt'
        logger.info("Saving checkpoint to {0}".format(filename))

#         with gzip.open(os.path.join(saveLocation, filename), 'w', compresslevel=5) as f:
#             data = (p, g, ancestry, ancestors, random.getstate())
#             pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
            
            
#         with gzip.open(os.path.join(saveLocation, 'train_test_data.pkl'), 'w', compresslevel=5) as f:
#             train_Test = (data_wrangler.X_train, data_wrangler.X_test, data_wrangler.y_train, data_wrangler.y_test)
#             pickle.dump(train_Test, f, protocol=pickle.HIGHEST_PROTOCOL)
    